In [39]:
import pandas as pd
import xarray as xr
import plotly.graph_objects as go
import plotly.express as px

## CMEMS layer depth


In [40]:
# xr.load_dataset(
#     "/Users/ash/Documents/Workspaces/Data/phd/HOT/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i_1718150094521.nc"
# )
cmems = xr.load_dataset(
    "/Users/ash/Documents/Workspaces/Data/phd/HOT/CMEMS/cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_1718149967802.nc"
)

In [41]:
layers_depth_xr = cmems["pelagic_layer_depth"].sel(
    latitude=22.75, longitude=-158.0, method="nearest"
)
layers_depth_xr

<xarray.DataArray 'pelagic_layer_depth' (time: 9131, depth: 3)> Size: 110kB
array([[117.25811 , 351.76886 , 820.79034 ],
       [120.61105 , 361.8427  , 844.306   ],
       [116.23569 , 348.70157 , 813.6484  ],
       ...,
       [126.50502 , 379.52463 , 885.56384 ],
       [129.85797 , 389.56842 , 908.9893  ],
       [124.610535, 373.81107 , 872.24225 ]], dtype=float32)
Coordinates:
  * depth      (depth) float32 12B 1.0 2.0 3.0
    latitude   float32 4B 22.75
    longitude  float32 4B -158.0
  * time       (time) datetime64[ns] 73kB 1998-01-01 1998-01-02 ... 2022-12-31
Attributes:
    units:          m
    standard_name:  sea_water_pelagic_layer_bottom_depth
    long_name:      Sea water pelagic layer bottom depth

In [42]:
layers_depth = (
    layers_depth_xr.to_dataframe().reset_index().drop(columns=["latitude", "longitude"])
)
depth_categories = {
    1.0: "epipelagic",
    2.0: "upper_mesopelagic",
    3.0: "lower_mesopelagic",
}
layers_depth["depth"] = layers_depth["depth"].map(depth_categories)
layers_depth

,time,depth,pelagic_layer_depth
0,1998-01-01,epipelagic,117.258110
1,1998-01-01,upper_mesopelagic,351.768860
2,1998-01-01,lower_mesopelagic,820.790344
3,1998-01-02,epipelagic,120.611053
4,1998-01-02,upper_mesopelagic,361.842712
...,...,...,...
27388,2022-12-30,upper_mesopelagic,389.568420
27389,2022-12-30,lower_mesopelagic,908.989319
27390,2022-12-31,epipelagic,124.610535
27391,2022-12-31,upper_mesopelagic,373.811066


## HOT observation


In [43]:
raw_data = pd.read_csv("../1_raw/hot_zooplankton.csv", index_col=0)

In [44]:
raw_data["time"] = pd.to_datetime(raw_data["time"])
raw_data = raw_data.drop(columns=["cruise"])

In [45]:
raw_data["is_day"] = raw_data.time.dt.hour.isin(range(6, 18))
raw_data["week"] = raw_data["time"].dt.to_period("W").dt.to_timestamp()
raw_data

,time,lat,lon,depth,frac,tow,vol,svol,wwt,dwt,carb,nit,abnd,is_day,week
0,1994-02-17 11:08:00,22.75,-158,181,0,8,726,NaN,0.2642,0.0132,3.95,1.09,24.0,True,1994-02-14
1,1994-02-17 11:08:00,22.75,-158,181,1,8,726,NaN,0.4170,0.0417,14.05,3.26,221.0,True,1994-02-14
2,1994-02-17 11:08:00,22.75,-158,181,2,8,726,NaN,0.2302,0.0230,8.89,2.19,240.0,True,1994-02-14
3,1994-02-17 11:08:00,22.75,-158,181,3,8,726,NaN,0.3738,0.0374,14.21,3.24,2331.0,True,1994-02-14
4,1994-02-17 11:08:00,22.75,-158,181,4,8,726,NaN,0.2749,0.0302,11.81,2.66,5595.0,True,1994-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9343,2022-09-02 22:13:00,22.75,-158,241,1,1640,135,NaN,2.9467,0.4162,135.35,34.69,NaN,False,2022-08-29
9344,2022-09-02 22:13:00,22.75,-158,241,2,1640,135,NaN,2.1526,0.3467,118.38,30.20,NaN,False,2022-08-29
9345,2022-09-02 22:13:00,22.75,-158,241,3,1640,135,NaN,2.5806,0.4098,143.47,35.31,NaN,False,2022-08-29
9346,2022-09-02 22:13:00,22.75,-158,241,4,1640,135,NaN,1.8745,0.3151,116.69,27.91,NaN,False,2022-08-29


## Plot


In [46]:
fig = px.line(
    layers_depth,
    x="time",
    y="pelagic_layer_depth",
    color="depth",
    title="Comparison of Seapodym Layers Depth and Observations in HOT",
)


fig.add_trace(
    go.Box(
        x=raw_data["week"],
        y=raw_data["depth"],
        marker=dict(color="black"),
        name="Observations Depth",
    )
)

# add a dash line between mean depth of the raw data
mean_depth = raw_data.groupby("week")["depth"].mean().reset_index()
fig.add_trace(
    go.Scatter(
        x=mean_depth["week"],
        y=mean_depth["depth"],
        mode="lines",
        line=dict(color="grey", dash="dash"),
        name="Mean Observations Depth",
    )
)


# swap the y-axis top and bottom
fig.update_yaxes(autorange="reversed")
# update size
fig.update_layout(height=800)
# update y label
fig.update_yaxes(title="Depth (m)")

fig.show()

In [50]:
fig.write_html("HOT_layer_and_obs_depths.html")